## Install dependencies and setting up GPU enviornment

In [1]:
!pip install tensorflow==2.0.0-beta1 
!pip install tqdm

     |████████████████████████████████| 87.9MB 48.9MB/s 
  Found existing installation: tensorflow 1.14.0
    Uninstalling tensorflow-1.14.0:
      Successfully uninstalled tensorflow-1.14.0


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!wget --no-check-certificate \
  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
  -O ./cats_and_dogs_filtered.zip

--2019-08-06 14:09:38--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 2607:f8b0:400e:c08::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M   125MB/s    in 0.5s    

2019-08-06 14:09:39 (125 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



## Dataset Preprocessing

### Import project dependenceies

In [4]:
import os
import zipfile 
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.0.0-beta1'

### Unzipping the Dogs vs Cats dataset
  

In [0]:
dataset_path = './cats_and_dogs_filtered.zip'

In [0]:
zip_object = zipfile.ZipFile(file=dataset_path, mode='r')

In [0]:
zip_object.extractall("./")

In [0]:
zip_object.close()

### Setting up dataset paths

In [0]:
dataset_path_new = "./cats_and_dogs_filtered/"

In [0]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

## Building the model

### Loading the pre-trained model (MobileNetV2)

In [0]:
IMG_SHAPE = (128, 128, 3)

In [12]:
base_model = tf.keras.applications.MobileNetV2(input_shape = IMG_SHAPE, include_top=False, weights = "imagenet")

9412608/9406464 [==============================] - 1s 0us/step


In [13]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

### Freezing the base model

In [0]:
base_model.trainable = False

### Defining the custom head for our network

In [15]:
base_model.output

<tf.Tensor 'out_relu/Identity:0' shape=(None, 4, 4, 1280) dtype=float32>

In [0]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [17]:
global_average_layer

<tf.Tensor 'global_average_pooling2d/Identity:0' shape=(None, 1280) dtype=float32>

In [0]:
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

### Defining the model

In [0]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [20]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
______________________________________________________________________________________________

### Compiling the model

In [0]:
model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy",metrics=["accuracy"])

### Creating Data Generators
Resizing images
    
    Big pre-trained architecture support only certain input sizes.
    
for example: MobileNet (architecture that we use) supports (96,96),(128,128),160,160),(192,192),(224,224).

In [0]:
data_gen_train = ImageDataGenerator(rescale = 1/255.)
data_gen_valid = ImageDataGenerator(rescale = 1/255.)

In [23]:
train_generator =data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128,class_mode ="binary")

Found 2000 images belonging to 2 classes.


In [24]:
valid_generator =data_gen_train.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128,class_mode ="binary")

Found 1000 images belonging to 2 classes.


### Training the model

In [25]:
model.fit_generator(train_generator, epochs = 5,validation_data=valid_generator)

Epoch 1/5


W0806 14:09:53.676434 140319793571712 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


16/16 [==============================] - 175s 11s/step - loss: 0.6550 - accuracy: 0.6245 - val_loss: 0.5237 - val_accuracy: 0.7340
Epoch 2/5
16/16 [==============================] - 99s 6s/step - loss: 0.5877 - accuracy: 0.6820 - val_loss: 0.4924 - val_accuracy: 0.7560
Epoch 3/5
16/16 [==============================] - 78s 5s/step - loss: 0.5451 - accuracy: 0.7260 - val_loss: 0.4622 - val_accuracy: 0.7880
Epoch 4/5
16/16 [==============================] - 77s 5s/step - loss: 0.5105 - accuracy: 0.7495 - val_loss: 0.4287 - val_accuracy: 0.8040
Epoch 5/5
16/16 [==============================] - 77s 5s/step - loss: 0.4721 - accuracy: 0.7815 - val_loss: 0.4078 - val_accuracy: 0.8220


### Transfer learning model evaluation

In [0]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [27]:
print(f"accuracy after Transfer Learning: {valid_accuracy*100}%")

accuracy after Transfer Learning: 82.20000267028809%


## Fine Tuning

there are a few pointers: 
  * DO NOT use Fine tuning on the whole network; only a few top layers are enough. In most cases, they are more specialized. the goal of fine-tuning is to adopt that specific part of the network for our custom (new) dataset.
  * Start with the fine tuning AFTER you have finished with the transfer learning step. If we try to perfom Fine tuning immedieately, gradients will be much different between our custom head layer and a few unfrozen layers from the base model. 

### Un-freeze a few top layers from the model

In [0]:
base_model.trainable = True

In [29]:
print(f"number of layers in base model : {len(base_model.layers)}")

number of layers in base model : 155


In [0]:
fine_tune_at = 100

In [0]:
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

### Compiling the model for fine-tuning

In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
             loss='binary_crossentropy',
             metrics=['accuracy'])

### Fine Tuning

In [33]:
model.fit_generator(train_generator,
                   epochs = 5,
                   validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - 120s 7s/step - loss: 0.1811 - accuracy: 0.9295 - val_loss: 0.1106 - val_accuracy: 0.9670
Epoch 2/5
16/16 [==============================] - 104s 6s/step - loss: 0.0309 - accuracy: 0.9955 - val_loss: 0.1044 - val_accuracy: 0.9700
Epoch 3/5
16/16 [==============================] - 105s 7s/step - loss: 0.0105 - accuracy: 1.0000 - val_loss: 0.1375 - val_accuracy: 0.9650
Epoch 4/5
16/16 [==============================] - 104s 6s/step - loss: 0.0049 - accuracy: 1.0000 - val_loss: 0.2031 - val_accuracy: 0.9510
Epoch 5/5
16/16 [==============================] - 106s 7s/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.2561 - val_accuracy: 0.9450


### Evaluating the fine tuned model

In [0]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [36]:
print(f"Validation accuracy after fine tuning : {valid_accuracy*100}%")

Validation accuracy after fine tuning : 94.49999928474426%
